In [5]:
import pandas as pd

import yaml

with open("../config.yaml", "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

config




{'input_data': {'marathon_data': '../data/raw/Berlin_Marathon_data_1974_2019.csv',
  'marathon_winners': '../data/raw/berlin_marathon_winners_1974_2024.csv'},
 'output_data': {'cleaned_data': '../data/clean/cleaned_marathon.csv',
  'cleaned_winners': '../data/clean/cleaned_marathon_winners.csv'}}

In [7]:
df = pd.read_csv(config["input_data"]["marathon_data"], low_memory=False)
print("Shape:", df.shape)
df.head()

Shape: (884944, 5)


,YEAR,COUNTRY,GENDER,AGE,TIME
0,1974,NaN,male,L1,02:44:53
1,1974,NaN,male,L2,02:46:43
2,1974,NaN,male,L2,02:48:08
3,1974,NaN,male,L,02:48:40
4,1974,NaN,male,L1,02:49:01


In [8]:
# Step 3 — Normalize column names to snake_case
before = df.columns.tolist()

df.columns = (
    df.columns
      .str.strip()                # remove leading/trailing spaces
      .str.lower()                # lower case
      .str.replace(' ', '_')      # spaces -> underscores
      .str.replace(r'[^a-z0-9_]', '', regex=True)  # drop weird chars
)

print("Before:", before)
print("After: ", df.columns.tolist())
df.head()


Before: ['YEAR', 'COUNTRY', 'GENDER', 'AGE', 'TIME']
After:  ['year', 'country', 'gender', 'age', 'time']


,year,country,gender,age,time
0,1974,NaN,male,L1,02:44:53
1,1974,NaN,male,L2,02:46:43
2,1974,NaN,male,L2,02:48:08
3,1974,NaN,male,L,02:48:40
4,1974,NaN,male,L1,02:49:01


In [13]:
# Step 5 — Check missing values
nulls = df.isna().sum()
print("Missing values per column:\n", nulls)

nulls_percent = (df.isna().mean() * 100).round(2)
print("\nPercentage of missing values per column:\n", nulls_percent)


Missing values per column:
 YEAR            0
COUNTRY    854148
GENDER          0
AGE         12838
TIME            0
dtype: int64

Percentage of missing values per column:
 YEAR        0.00
COUNTRY    96.52
GENDER      0.00
AGE         1.45
TIME        0.00
dtype: float64


In [15]:
# Step 6 — Clean 'country' and 'age'

# 1) Drop country (too many missing values ~96%)
if "COUNTRY" in df.columns:
    df = df.drop(columns=["COUNTRY"])
    print("Column 'COUNTRY' removed ✅")

# 2) Fill missing age values with 'Unknown'
if "age" in df.columns:
    missing_before = df["age"].isna().sum()
    df["age"] = df["age"].fillna("Unknown").astype(str).str.strip()
    missing_after = df["age"].isna().sum()
    print(f"'age' missing before: {missing_before}, after fill: {missing_after}")

# Quick check
df.head()


Column 'COUNTRY' removed ✅


,YEAR,GENDER,AGE,TIME
0,1974,male,L1,02:44:53
1,1974,male,L2,02:46:43
2,1974,male,L2,02:48:08
3,1974,male,L,02:48:40
4,1974,male,L1,02:49:01


In [16]:
# Step 7 — Clean 'GENDER'

if "GENDER" in df.columns:
    print("Unique values before:", df["GENDER"].unique())

    df["GENDER"] = (
        df["GENDER"].astype(str).str.lower().str.strip()
        .map({"m":"male", "male":"male", 
              "f":"female", "female":"female"})
        .fillna("unknown")
    )

    print("Unique values after:", df["GENDER"].unique())
    print(df["GENDER"].value_counts())


Unique values before: ['male' 'female']
Unique values after: ['male' 'female']
GENDER
male      726467
female    158477
Name: count, dtype: int64


In [21]:
# Step 8 — Parse TIME to duration and seconds

# 1) elegir la columna de tiempo (respeta mayúsculas)
time_col = "TIME" if "TIME" in df.columns else [c for c in df.columns if "time" in c.lower()][0]
print("Usando columna de tiempo:", time_col)

# 2) convert to timedelta (NaT if value is bad)
df["finish_time"] = pd.to_timedelta(df[time_col], errors="coerce")

# 3) create seconds (float)
df["finish_seconds"] = df["finish_time"].dt.total_seconds()

# 4) quick verification
print("NaT en finish_time:", df["finish_time"].isna().sum())
df[[time_col, "finish_time", "finish_seconds"]].head()


Usando columna de tiempo: TIME
NaT en finish_time: 2405


,TIME,finish_time,finish_seconds
0,02:44:53,0 days 02:44:53,9893.0
1,02:46:43,0 days 02:46:43,10003.0
2,02:48:08,0 days 02:48:08,10088.0
3,02:48:40,0 days 02:48:40,10120.0
4,02:49:01,0 days 02:49:01,10141.0


In [24]:

# Estimated mapping (common in marathons, but not official for Berlin dataset)

age_mapping = {
    "L1": "18-24",
    "L2": "25-29",
    "L3": "30-34",
    "L4": "35-39",
    "L5": "40-44",
    "L6": "45-49",
    "L7": "50-54",
    "L8": "55-59",
    "L9": "60-64",
    "L10": "65-69",
    "L11": "70-74",
    "L12": "75+"
}

# Apply mapping
df["AGE_group"] = df["AGE"].map(age_mapping).fillna("Unknown")

df[["AGE", "AGE_group"]].head(15)


,AGE,AGE_group
0,L1,18-24
1,L2,25-29
2,L2,25-29
3,L,Unknown
4,L1,18-24
5,L,Unknown
6,L,Unknown
7,L2,25-29
8,L2,25-29
9,L1,18-24


In [26]:
# Step 10 — Map AGE codes directly to estimated 5-year groups

age_mapping = {
    "L1": "18-24",
    "L2": "25-29",
    "L3": "30-34",
    "L4": "35-39",
    "L5": "40-44",
    "L6": "45-49",
    "L7": "50-54",
    "L8": "55-59",
    "L9": "60-64",
    "L10": "65-69",
    "L11": "70-74",
    "L12": "75+"
}

df["AGE_group"] = df["AGE"].map(age_mapping).fillna("Unknown")

# Quick check
print(df[["AGE", "AGE_group"]].head(15))



   AGE AGE_group
0   L1     18-24
1   L2     25-29
2   L2     25-29
3    L   Unknown
4   L1     18-24
5    L   Unknown
6    L   Unknown
7   L2     25-29
8   L2     25-29
9   L1     18-24
10  L1     18-24
11  L2     25-29
12  L1     18-24
13  L1     18-24
14  L2     25-29


In [27]:
# Save cleaned & wrangled dataset
output_path = config["output_data"]["cleaned_data"]
df.to_csv(output_path, index=False)

print("Dataset saved to:", output_path)
print("Final shape:", df.shape)


Dataset saved to: ../data/clean/cleaned_marathon.csv
Final shape: (884944, 7)
